In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install catboost -q

     |████████████████████████████████| 66.1MB 86kB/s 


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import copy

In [ ]:
train_ = pd.read_csv( 'Train.csv')
test_ = pd.read_csv('Test.csv')
submission_ = pd.read_csv('SampleSubmission.csv')

In [ ]:
%%time
from sklearn.model_selection import KFold
fold_train = []
kf = KFold(n_splits=5, shuffle=False)
for r, (train_index, test_index) in enumerate(kf.split(train_)):
  test = train_.iloc[test_index]

  X_test = []
  X_test_columns = test.columns
  for v in test.values:
    info = v[:8]
    binary = v[8:]
    index = [k for k, i in enumerate(binary) if i == 1]
    for i in index:
      for k in range(len(binary)):
        if k == i:
          binary_transformed = list(copy.copy(binary))
          binary_transformed[i] = 0
          X_test.append(list(info) + binary_transformed + [8+k])

  X_test = pd.DataFrame(X_test)
  X_test.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
          'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
          '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
          'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'product_num']

  X_test['ID'] = [str(r)+'_'+str(i) for i in range(X_test.shape[0])]

  test = []
  test_columns = X_test.columns

  for v in X_test.values:
    info = v[:8]
    binary = v[8:-1]
    num = v[-1]
    index = [k for k, i in enumerate(binary) if i == 1]
    for k in range(len(binary)):
        if k not in index:
          if 8+k == num:
            test.append(list(info) + list(binary) + [test_columns[8+k]] + [1])
          else:
            test.append(list(info) + list(binary) + [test_columns[8+k]] + [0])
  test = pd.DataFrame(test)
  test.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
        'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
        '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
        'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'product_pred', 'target']

  test = test[['ID', 'product_pred', 'target']]
  test['ID X PCODE'] = test['ID'] + ' X ' + test['product_pred']
  del test['ID'], test['product_pred']
  test.columns = ['Label', 'ID X PCODE']
  
  path_0 = '/content/drive/My Drive/Zimnat/stack/'
  path_1 = '/content/drive/My Drive/Zimnat/STACK_LGBM_BASELINE_2/'
  
  path_2 = '/content/drive/My Drive/Zimnat/stack_LIGHTGBM_BASE_3/'
  
  path_4 = '/content/drive/My Drive/Zimnat/Stack_XGBOOST_RONNY/'
  path_5 = '/content/drive/My Drive/Zimnat/CNN_verification/'
  path_6 = '/content/drive/My Drive/Zimnat/FFN_verification/'
  path_7 = '/content/drive/My Drive/Zimnat/Stack_LGBM_RONNY/'
  

  submit1 = pd.read_csv(path_0 + '0_fold' + str(r) + '.csv')
  submit2 = pd.read_csv( path_1 +'1_fold' + str(r) + '.csv')
  submit3 = pd.read_csv(path_2 + '2_fold' + str(r) + '.csv')
  submit4 = pd.read_csv(path_4 + '0_fold' + str(r) + '.csv')
  submit5 = pd.read_csv(path_5 + '0_fold' + str(r) + '.csv')
  submit6 = pd.read_csv(path_6 + '0_fold' + str(r) + '.csv')
  submit7 = pd.read_csv(path_7 + '0_fold' + str(r) + '.csv')


  test = test.merge(submit1, how='left', on='ID X PCODE')
  test = test.merge(submit2, how='left', on='ID X PCODE')
  test = test.merge(submit3, how='left', on='ID X PCODE')
  test = test.merge(submit4, how='left', on='ID X PCODE')
  test = test.merge(submit5, how='left', on='ID X PCODE')
  test = test.merge(submit6, how='left', on='ID X PCODE')
  test = test.merge(submit7, how='left', on='ID X PCODE')



  test.columns = ['target', 'ID X PCODE', 'Label1', 
                  'Label2', 
                  'Label3','Label4','Label5','Label6','Label7']

  fold_train.append(test)

fold_train = pd.concat(fold_train)

CPU times: user 35.6 s, sys: 1.48 s, total: 37.1 s
Wall time: 59.6 s


In [ ]:
test = []
test_columns = test_.columns

for v in test_.values:
  info = v[:8]
  binary = v[8:]
  index = [k for k, i in enumerate(binary) if i == 1]
  for k in range(len(binary)):
    if k not in index:
      test.append(list(info) + list(binary) + [test_columns[8+k]])
test = pd.DataFrame(test)
test.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
        'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
        '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
        'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'product_pred',]

test = test[['ID', 'product_pred']]
test['ID X PCODE'] = test['ID'] + ' X ' + test['product_pred']
del test['ID'], test['product_pred']

path_0 = '/content/drive/My Drive/Zimnat/stack/'
path_1 = '/content/drive/My Drive/Zimnat/STACK_LGBM_BASELINE_2/'
path_2 = '/content/drive/My Drive/Zimnat/stack_LIGHTGBM_BASE_3/'

path_4 = '/content/drive/My Drive/Zimnat/Stack_XGBOOST_RONNY/'
path_5 = '/content/drive/My Drive/Zimnat/CNN_verification/'
path_6 = '/content/drive/My Drive/Zimnat/FFN_verification/'
path_7 = '/content/drive/My Drive/Zimnat/Stack_LGBM_RONNY/'


submit1 = pd.read_csv(path_0+ '0_main.csv') 
submit2 = pd.read_csv(path_1 + '1_main.csv')
submit3 = pd.read_csv(path_2+ '2_main.csv')
submit4 = pd.read_csv(path_4 + '0_main.csv')
submit5 = pd.read_csv(path_5 + '0_main.csv' )
submit6 = pd.read_csv(path_6 + '0_main.csv' )
submit7 = pd.read_csv(path_7 + '0_main.csv' )

test = test.merge(submit1, how='left', on='ID X PCODE')
test = test.merge(submit2, how='left', on='ID X PCODE')
test = test.merge(submit3, how='left', on='ID X PCODE')
test = test.merge(submit4, how='left', on='ID X PCODE')
test = test.merge(submit5, how='left', on='ID X PCODE')
test = test.merge(submit6, how='left', on='ID X PCODE')
test = test.merge(submit7, how='left', on='ID X PCODE')

test.columns = ['ID X PCODE', 'Label1',
                'Label2',
                'Label3','Label4','Label5','Label6','Label7']

In [ ]:
fold_train.head()

,target,ID X PCODE,Label1,Label2,Label3,Label4,Label5,Label6,Label7
0,0,0_0 X P5DA,0.000002,0.000007,0.000104,1.736310e-08,1.666908e-16,1.147360e-09,2.877702e-08
1,0,0_0 X RIBP,0.000040,0.000067,0.001975,2.170020e-05,1.672458e-11,5.417373e-09,5.373559e-05
2,0,0_0 X 8NN1,0.000001,0.000004,0.000066,2.628379e-07,4.753043e-13,4.251525e-13,3.275651e-07
3,0,0_0 X 7POT,0.000007,0.000003,0.000071,9.227806e-07,1.920112e-13,1.147974e-08,2.244441e-06
4,0,0_0 X 66FJ,0.000007,0.000003,0.000082,1.527384e-06,7.657072e-13,1.536300e-10,1.889719e-06


In [ ]:
test.head()

,ID X PCODE,Label1,Label2,Label3,Label4,Label5,Label6,Label7
0,F86J5PC X P5DA,0.000002,0.000106,0.000351,4.981359e-07,0.000020,0.000009,0.000001
1,F86J5PC X RIBP,0.000395,0.000344,0.002852,5.070527e-04,0.000392,0.000355,0.000238
2,F86J5PC X 8NN1,0.000003,0.000065,0.000044,2.708316e-06,0.000002,0.000004,0.000004
3,F86J5PC X 7POT,0.000071,0.000061,0.000097,5.838094e-05,0.000015,0.000042,0.000044
4,F86J5PC X 66FJ,0.000073,0.000050,0.000036,1.086586e-04,0.000059,0.000060,0.000023


new

In [ ]:
fold_train['ID'] = fold_train['ID X PCODE'].apply(lambda x: x.split(' X ')[0])
fold_train['PCODE'] = fold_train['ID X PCODE'].apply(lambda x: x.split(' X ')[1])

test['ID'] = test['ID X PCODE'].apply(lambda x: x.split(' X ')[0])
test['PCODE'] = test['ID X PCODE'].apply(lambda x: x.split(' X ')[1])

del fold_train['ID X PCODE'], test['ID X PCODE']

In [ ]:
new_df = pd.DataFrame(np.zeros((fold_train['ID'].nunique(), fold_train['PCODE'].nunique()*7+1)))
new_df.index = fold_train['ID'].unique()
new_df.columns = list(fold_train['PCODE'].unique() + '_0') + list(fold_train['PCODE'].unique() + '_1') + list(fold_train['PCODE'].unique() + '_2') + list(fold_train['PCODE'].unique() + '_3') + list(fold_train['PCODE'].unique() + '_4') + list(fold_train['PCODE'].unique() + '_5') + list(fold_train['PCODE'].unique() + '_6') + ['target']

In [ ]:
fold_train.values[1]

array([0, 3.998375188628589e-05, 6.685188094018476e-05,
       0.0019753662621946745, 2.17001997953048e-05,
       1.6724578319471384e-11, 5.417372772598128e-09,
       5.373558615221109e-05, '0_0', 'RIBP'], dtype=object)

In [12]:
for v in tqdm(fold_train.values):
   new_df.loc[v[8], v[9]+'_0'] = v[1]
   new_df.loc[v[8], v[9]+'_1'] = v[2]
   new_df.loc[v[8], v[9]+'_2'] = v[3]
   new_df.loc[v[8], v[9]+'_3'] = v[4]
   new_df.loc[v[8], v[9]+'_4'] = v[5]
   new_df.loc[v[8], v[9]+'_5'] = v[6]
   new_df.loc[v[8], v[9]+'_6'] = v[7]



   if v[0] == 1:
     new_df.loc[v[8], 'target'] = v[9]

100%|██████████| 1298055/1298055 [2:03:18<00:00, 175.45it/s]


In [13]:
new_df_test = pd.DataFrame(np.zeros((test['ID'].nunique(), test['PCODE'].nunique()*7)))
new_df_test.index = test['ID'].unique()
new_df_test.columns = list(test['PCODE'].unique() + '_0') + list(test['PCODE'].unique() + '_1') + list(test['PCODE'].unique() + '_2') + list(test['PCODE'].unique() + '_3') + list(test['PCODE'].unique() + '_4') + list(test['PCODE'].unique() + '_5') + list(test['PCODE'].unique() + '_6')  

In [14]:
test

,Label1,Label2,Label3,Label4,Label5,Label6,Label7,ID,PCODE
0,0.000002,0.000106,0.000351,4.981359e-07,2.025166e-05,8.900241e-06,1.261470e-06,F86J5PC,P5DA
1,0.000395,0.000344,0.002852,5.070527e-04,3.919043e-04,3.546129e-04,2.377628e-04,F86J5PC,RIBP
2,0.000003,0.000065,0.000044,2.708316e-06,1.790804e-06,3.968615e-06,4.304941e-06,F86J5PC,8NN1
3,0.000071,0.000061,0.000097,5.838094e-05,1.519518e-05,4.216255e-05,4.373406e-05,F86J5PC,7POT
4,0.000073,0.000050,0.000036,1.086586e-04,5.875205e-05,6.021684e-05,2.282244e-05,F86J5PC,66FJ
...,...,...,...,...,...,...,...,...,...
197142,0.000008,0.000032,0.000425,9.857251e-07,4.623162e-12,9.422341e-18,2.842085e-07,WHII93K,JWFN
197143,0.000022,0.000019,0.012963,2.741064e-09,5.111000e-13,1.471686e-09,3.363596e-07,WHII93K,JZ9D
197144,0.000025,0.000014,0.008980,2.174445e-08,1.927185e-14,3.919759e-15,3.358848e-07,WHII93K,J9JW
197145,0.000174,0.000123,0.004435,1.356621e-05,4.494113e-11,2.499644e-08,3.914231e-05,WHII93K,GHYX


In [15]:
for v in tqdm(test.values):
   new_df_test.loc[v[7], v[8]+'_0'] = v[0]
   new_df_test.loc[v[7], v[8]+'_1'] = v[1]
   new_df_test.loc[v[7], v[8]+'_2'] = v[2]
   new_df_test.loc[v[7], v[8]+'_3'] = v[3]
   new_df_test.loc[v[7], v[8]+'_4'] = v[4]
   new_df_test.loc[v[7], v[8]+'_5'] = v[5]
   new_df_test.loc[v[7], v[8]+'_6'] = v[6]

100%|██████████| 197147/197147 [04:13<00:00, 777.30it/s]


In [16]:
new_df_test

,P5DA_0,RIBP_0,8NN1_0,7POT_0,66FJ_0,GYSR_0,SOP4_0,PYUQ_0,LJR9_0,N2MW_0,AHXO_0,BSTQ_0,FM3X_0,K6QO_0,QBOL_0,JWFN_0,JZ9D_0,J9JW_0,GHYX_0,ECY3_0,RVSZ_0,P5DA_1,RIBP_1,8NN1_1,7POT_1,66FJ_1,GYSR_1,SOP4_1,PYUQ_1,LJR9_1,N2MW_1,AHXO_1,BSTQ_1,FM3X_1,K6QO_1,QBOL_1,JWFN_1,JZ9D_1,J9JW_1,GHYX_1,...,8NN1_5,7POT_5,66FJ_5,GYSR_5,SOP4_5,PYUQ_5,LJR9_5,N2MW_5,AHXO_5,BSTQ_5,FM3X_5,K6QO_5,QBOL_5,JWFN_5,JZ9D_5,J9JW_5,GHYX_5,ECY3_5,RVSZ_5,P5DA_6,RIBP_6,8NN1_6,7POT_6,66FJ_6,GYSR_6,SOP4_6,PYUQ_6,LJR9_6,N2MW_6,AHXO_6,BSTQ_6,FM3X_6,K6QO_6,QBOL_6,JWFN_6,JZ9D_6,J9JW_6,GHYX_6,ECY3_6,RVSZ_6
F86J5PC,2.038956e-06,0.000395,3.067319e-06,0.000071,0.000073,1.408622e-06,0.000324,0.001754,0.000043,0.000104,0.000025,0.000041,5.788468e-06,0.981331,0.010432,1.969000e-04,0.000121,0.000117,0.002254,0.002703,0.000000,0.000106,0.000344,6.485056e-05,0.000061,4.950373e-05,0.000102,0.000136,0.000946,0.000468,0.000398,0.000137,0.000309,0.000309,0.991898,0.006118,0.000337,0.000286,0.000213,0.004326,...,3.968615e-06,4.216255e-05,6.021684e-05,8.734292e-09,7.500620e-05,8.534630e-04,9.530735e-05,7.624227e-04,8.522003e-04,6.895784e-07,6.489738e-08,9.731721e-01,1.146844e-02,2.909856e-04,2.615183e-07,1.665736e-09,9.176350e-03,2.782772e-03,0.000000e+00,1.261470e-06,0.000238,4.304941e-06,4.373406e-05,2.282244e-05,6.843454e-08,1.582831e-04,0.002231,9.142753e-05,0.000237,0.000049,4.302263e-05,2.240519e-06,0.967394,0.016627,3.759664e-05,1.691602e-05,1.686431e-05,0.005573,0.006982,0.000000
H6141K3,4.533728e-05,0.001618,1.338280e-04,0.000423,0.000722,5.054484e-06,0.000852,0.010072,0.000747,0.003395,0.001391,0.000605,2.223099e-04,0.000000,0.490467,1.635593e-01,0.000821,0.000804,0.148844,0.172166,0.000000,0.000561,0.001567,1.538794e-04,0.000618,6.337112e-04,0.000652,0.000227,0.009206,0.002240,0.001738,0.000361,0.001024,0.005218,0.000000,0.342929,0.020726,0.002023,0.001780,0.267824,...,2.885594e-04,3.894453e-04,2.889341e-03,3.878320e-07,2.581501e-03,4.503523e-03,1.990658e-03,2.598647e-03,1.897261e-03,1.249779e-06,3.407637e-07,0.000000e+00,4.640821e-01,4.991151e-02,1.757511e-05,1.535072e-08,2.931938e-01,1.692053e-01,0.000000e+00,1.980153e-05,0.001973,1.005583e-04,1.166777e-03,2.276870e-04,8.167944e-07,4.790965e-04,0.015801,1.601321e-03,0.000728,0.000432,2.529557e-04,5.646597e-05,0.000000,0.493606,2.984981e-02,2.599137e-04,2.580132e-04,0.318413,0.129049,0.000000
RBAYUXZ,3.768160e-05,0.002367,2.048250e-04,0.000194,0.000683,2.046647e-06,0.000811,0.006395,0.001105,0.006647,0.004289,0.000109,6.722619e-06,0.000000,0.879228,1.735700e-02,0.000544,0.000238,0.008176,0.070898,0.000000,0.000685,0.006903,1.087682e-03,0.000818,3.137980e-03,0.000290,0.002362,0.015840,0.003634,0.019590,0.002467,0.001954,0.000867,0.000000,0.789709,0.015551,0.001891,0.000849,0.019131,...,1.659549e-03,5.433824e-04,8.259876e-03,1.049585e-05,4.768267e-03,1.437572e-02,1.072309e-03,6.429023e-03,1.464969e-03,2.346057e-06,4.317966e-07,0.000000e+00,7.626024e-01,1.039652e-02,5.568569e-04,6.552045e-09,1.274711e-02,1.470469e-01,0.000000e+00,5.023610e-06,0.002617,3.440755e-04,2.080012e-04,6.609540e-04,3.605870e-07,1.419639e-03,0.004716,7.782638e-04,0.011002,0.001992,6.655856e-05,6.649077e-06,0.000000,0.887562,1.371175e-02,5.899922e-05,5.854559e-05,0.010211,0.063187,0.000000
KCBILBQ,1.977373e-06,0.000176,1.649559e-06,0.000035,0.000036,3.905146e-07,0.000191,0.000977,0.000152,0.000435,0.000251,0.000040,2.931731e-06,0.985894,0.005515,1.189624e-04,0.000081,0.000117,0.003064,0.002905,0.000000,0.000079,0.000245,3.854714e-05,0.000041,3.876904e-05,0.000029,0.000205,0.000814,0.000402,0.000518,0.000151,0.000153,0.000081,0.991719,0.004782,0.000163,0.000119,0.000090,0.002589,...,1.388489e-06,4.232597e-06,5.220904e-06,7.987845e-10,6.158727e-06,8.905758e-05,1.236993e-05,1.163326e-03,6.681437e-04,3.000490e-07,1.972946e-09,9.898165e-01,2.605387e-03,3.019363e-05,1.228935e-07,1.877503e-08,4.488842e-03,1.064755e-03,0.000000e+00,4.957798e-07,0.000222,4.151966e-06,5.939825e-05,2.514319e-05,3.459391e-08,4.743796e-04,0.000907,1.813953e-04,0.000530,0.000381,2.305240e

In [17]:
new_df

,P5DA_0,RIBP_0,8NN1_0,7POT_0,66FJ_0,GYSR_0,SOP4_0,RVSZ_0,PYUQ_0,LJR9_0,N2MW_0,AHXO_0,BSTQ_0,FM3X_0,QBOL_0,JWFN_0,JZ9D_0,J9JW_0,GHYX_0,ECY3_0,K6QO_0,P5DA_1,RIBP_1,8NN1_1,7POT_1,66FJ_1,GYSR_1,SOP4_1,RVSZ_1,PYUQ_1,LJR9_1,N2MW_1,AHXO_1,BSTQ_1,FM3X_1,QBOL_1,JWFN_1,JZ9D_1,J9JW_1,GHYX_1,...,7POT_5,66FJ_5,GYSR_5,SOP4_5,RVSZ_5,PYUQ_5,LJR9_5,N2MW_5,AHXO_5,BSTQ_5,FM3X_5,QBOL_5,JWFN_5,JZ9D_5,J9JW_5,GHYX_5,ECY3_5,K6QO_5,P5DA_6,RIBP_6,8NN1_6,7POT_6,66FJ_6,GYSR_6,SOP4_6,RVSZ_6,PYUQ_6,LJR9_6,N2MW_6,AHXO_6,BSTQ_6,FM3X_6,QBOL_6,JWFN_6,JZ9D_6,J9JW_6,GHYX_6,ECY3_6,K6QO_6,target
0_0,2.062916e-06,0.000040,1.329303e-06,0.000007,0.000007,1.562166e-07,0.000004,0.999314,0.000062,0.000012,0.000108,0.000050,0.000030,9.034984e-07,0.000090,0.000008,0.000020,0.000016,0.000182,0.000037,0.000000,0.000007,0.000067,0.000004,0.000003,0.000003,0.000005,0.000007,0.999873,0.000066,0.000022,0.000036,0.000011,0.000010,0.000005,0.000160,0.000036,0.000022,0.000016,0.000163,...,1.147974e-08,1.536300e-10,4.052474e-17,8.906488e-10,0.999981,5.281901e-07,5.869792e-08,1.694414e-05,2.714263e-07,2.934339e-10,2.755734e-12,6.139630e-07,1.741285e-14,9.412347e-09,1.138725e-16,5.912937e-07,5.043646e-08,0.000000,2.877702e-08,0.000054,3.275651e-07,2.244441e-06,0.000002,1.215748e-09,0.000002,0.999694,0.000023,0.000002,0.000047,0.000004,1.364562e-06,8.064520e-08,0.000056,7.912432e-07,7.300248e-07,7.172254e-07,0.000087,0.000014,0.000000,RVSZ
0_1,1.194800e-05,0.000173,3.812930e-06,0.000010,0.000021,3.163981e-07,0.000012,0.000000,0.000614,0.000043,0.000445,0.000158,0.000069,2.675490e-06,0.003556,0.000186,0.000047,0.000042,0.002238,0.000797,0.991568,0.000084,0.000554,0.000027,0.000027,0.000023,0.000041,0.000052,0.000000,0.001033,0.000398,0.000613,0.000078,0.000076,0.000056,0.004113,0.000281,0.000222,0.000148,0.004045,...,5.308498e-07,5.267993e-06,1.875081e-09,3.445359e-06,0.000000,5.281822e-05,2.712374e-06,1.656913e-04,2.256979e-05,6.197253e-08,1.550715e-10,1.142636e-03,1.857753e-05,9.331117e-08,1.244154e-08,2.443292e-03,1.762506e-04,0.995949,5.797709e-07,0.000949,5.884530e-06,3.418899e-05,0.000033,2.175808e-08,0.000023,0.000000,0.000555,0.000034,0.000824,0.000068,2.323625e-05,1.328613e-06,0.003708,2.175840e-04,1.299573e-05,1.269484e-05,0.002378,0.000468,0.990475,K6QO
0_2,1.050081e-06,0.000030,1.271614e-06,0.000010,0.000009,3.174679e-07,0.000013,0.998910,0.000145,0.000022,0.000075,0.000017,0.000007,8.675750e-07,0.000240,0.000002,0.000041,0.000019,0.000384,0.000067,0.000000,0.000010,0.000037,0.000005,0.000005,0.000005,0.000005,0.000010,0.999904,0.000080,0.000034,0.000043,0.000017,0.000013,0.000005,0.000253,0.000033,0.000024,0.000020,0.000129,...,1.595816e-07,2.500631e-10,4.240571e-16,2.114912e-09,0.999961,1.505756e-06,3.730126e-07,2.209410e-05,2.714393e-08,1.660969e-08,2.035463e-10,1.299568e-05,6.658820e-13,3.773392e-08,2.643902e-15,1.038446e-06,3.944733e-07,0.000000,3.224956e-08,0.000020,3.333607e-07,9.121229e-07,0.000001,1.369493e-09,0.000002,0.999691,0.000035,0.000005,0.000024,0.000009,8.499384e-07,6.975733e-08,0.000126,4.255521e-07,1.610616e-06,7.172233e-07,0.000052,0.000021,0.000000,RVSZ
0_3,6.748771e-06,0.000189,5.630120e-06,0.000017,0.000029,8.136376e-07,0.000039,0.000000,0.001783,0.000072,0.000286,0.000080,0.000025,1.899558e-06,0.012337,0.000038,0.000114,0.000053,0.004835,0.001153,0.978929,0.000096,0.000418,0.000033,0.000029,0.000036,0.000035,0.000068,0.000000,0.001277,0.000499,0.000561,0.000122,0.000093,0.000052,0.009613,0.000218,0.000195,0.000148,0.001819,...,8.549226e-06,4.557471e-05,8.154189e-09,9.919467e-06,0.000000,1.419311e-03,5.451842e-05,1.187164e-04,3.191569e-05,4.363407e-07,1.143065e-08,3.510761e-02,1.143659e-04,1.072589e-06,2.417389e-07,3.531630e-03,1.012926e-03,0.958345,7.534269e-07,0.000390,6.834462e-06,1.555265e-05,0.000026,3.090760e-08,0.000030,0.000000,0.000865,0.000077,0.000385,0.000198,1.629804e-05,1.139401e-06,0.011154,4.020223e-05,1.701115e-05,1.371342e-05,0.001278,0.000753,0.984544,K6QO
0_4,1.410049e-05,0.013857,1.315864e-03,0.000134,0.018212,1.173993e-05,0.936

In [18]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(new_df['target'])
le.transform(new_df['target'])

array([19, 12, 19, ..., 12, 19, 12])

In [19]:
new_df['target'] = le.transform(new_df['target'])

In [20]:
from catboost import CatBoostClassifier

models = []
models.append(CatBoostClassifier(random_state=17, max_depth = 3))
models.append(CatBoostClassifier(random_state=18, num_trees=700, max_depth=5))
models.append(CatBoostClassifier(random_state=19, num_trees=500, max_depth = 3))
models.append(CatBoostClassifier(random_state=20, num_trees=1200, max_depth=3))
#models.append(CatBoostClassifier(random_state=21))

In [21]:
for i in range(len(models)):
  models[i].fit(new_df.drop(columns=['target',]), new_df['target'])

Learning rate set to 0.098003
0:	learn: 1.8013797	total: 831ms	remaining: 13m 49s
1:	learn: 1.5775306	total: 1.56s	remaining: 13m
2:	learn: 1.4232560	total: 2.28s	remaining: 12m 38s
3:	learn: 1.3075719	total: 3s	remaining: 12m 27s
4:	learn: 1.2166544	total: 3.73s	remaining: 12m 22s
5:	learn: 1.1374799	total: 4.44s	remaining: 12m 16s
6:	learn: 1.0709099	total: 5.18s	remaining: 12m 15s
7:	learn: 1.0185155	total: 5.91s	remaining: 12m 12s
8:	learn: 0.9732143	total: 6.63s	remaining: 12m 9s
9:	learn: 0.9240546	total: 7.39s	remaining: 12m 12s
10:	learn: 0.8870212	total: 8.11s	remaining: 12m 9s
11:	learn: 0.8567780	total: 8.75s	remaining: 12m
12:	learn: 0.8225863	total: 9.37s	remaining: 11m 51s
13:	learn: 0.7941923	total: 10.1s	remaining: 11m 51s
14:	learn: 0.7702941	total: 10.7s	remaining: 11m 45s
15:	learn: 0.7487975	total: 11.3s	remaining: 11m 36s
16:	learn: 0.7282554	total: 11.9s	remaining: 11m 30s
17:	learn: 0.7058247	total: 12.5s	remaining: 11m 24s
18:	learn: 0.6908606	total: 13.1s	remai

In [22]:
df_answ = []
for num_pred in range(len(models)):
  proba = models[num_pred].predict_proba(new_df_test)
  y_test = pd.DataFrame(proba)
  y_test.columns = le.inverse_transform(y_test.columns)

  true_values = []
  for v in test_.values:
    binary = v[8:]
    index = [k for k, i in enumerate(binary) if i == 1]
    for k in test_.columns[8:][index]:
      true_values.append(v[0] + ' X ' + k)

  answer_mass = []
  for i in range(new_df_test.index.shape[0]):
    id = new_df_test.index[i]
    for c in y_test.columns:
      answer_mass.append([id + ' X ' + c, y_test[c].iloc[i]])

  df_answer = pd.DataFrame(answer_mass)
  df_answer.columns = ['ID X PCODE', 'Label']
  for i in range(df_answer.shape[0]):
    if df_answer['ID X PCODE'].iloc[i] in true_values:
      df_answer['Label'].iloc[i] = 1.0

  df_answ.append(df_answer)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [23]:
df_answ

[            ID X PCODE     Label
 0       F86J5PC X 66FJ  0.000031
 1       F86J5PC X 7POT  0.000023
 2       F86J5PC X 8NN1  0.000004
 3       F86J5PC X AHXO  0.000055
 4       F86J5PC X BSTQ  0.000009
 ...                ...       ...
 209995  WHII93K X PYUQ  0.000055
 209996  WHII93K X QBOL  0.000204
 209997  WHII93K X RIBP  0.000017
 209998  WHII93K X RVSZ  0.999576
 209999  WHII93K X SOP4  0.000003
 
 [210000 rows x 2 columns],             ID X PCODE         Label
 0       F86J5PC X 66FJ  3.073976e-05
 1       F86J5PC X 7POT  3.715490e-06
 2       F86J5PC X 8NN1  9.442298e-07
 3       F86J5PC X AHXO  2.537036e-05
 4       F86J5PC X BSTQ  3.095194e-06
 ...                ...           ...
 209995  WHII93K X PYUQ  8.928455e-06
 209996  WHII93K X QBOL  1.087362e-04
 209997  WHII93K X RIBP  3.618704e-06
 209998  WHII93K X RVSZ  9.998433e-01
 209999  WHII93K X SOP4  2.179412e-07
 
 [210000 rows x 2 columns],             ID X PCODE     Label
 0       F86J5PC X 66FJ  0.000048
 1       F

In [24]:
df_answ_labels = []
for v in df_answ:
  df_answ_labels.append(v['Label'])

In [25]:
df_answ_labels

[0         0.000031
 1         0.000023
 2         0.000004
 3         0.000055
 4         0.000009
             ...   
 209995    0.000055
 209996    0.000204
 209997    0.000017
 209998    0.999576
 209999    0.000003
 Name: Label, Length: 210000, dtype: float64, 0         3.073976e-05
 1         3.715490e-06
 2         9.442298e-07
 3         2.537036e-05
 4         3.095194e-06
               ...     
 209995    8.928455e-06
 209996    1.087362e-04
 209997    3.618704e-06
 209998    9.998433e-01
 209999    2.179412e-07
 Name: Label, Length: 210000, dtype: float64, 0         0.000048
 1         0.000036
 2         0.000006
 3         0.000071
 4         0.000020
             ...   
 209995    0.000059
 209996    0.000132
 209997    0.000019
 209998    0.999630
 209999    0.000003
 Name: Label, Length: 210000, dtype: float64, 0         0.000045
 1         0.000020
 2         0.000005
 3         0.000081
 4         0.000009
             ...   
 209995    0.000046
 209996    0.000152
 

In [26]:
np.mean(df_answ_labels, axis=0)

array([3.84983979e-05, 2.07645102e-05, 4.11374200e-06, ...,
       1.25927664e-05, 9.99677304e-01, 1.66655482e-06])

In [27]:
df_answer = df_answ[0]
df_answer['Label'] = np.mean(df_answ_labels, axis=0)

In [30]:
df_answer

,ID X PCODE,Label
0,F86J5PC X 66FJ,0.000038
1,F86J5PC X 7POT,0.000021
2,F86J5PC X 8NN1,0.000004
3,F86J5PC X AHXO,0.000058
4,F86J5PC X BSTQ,0.000010
...,...,...
209995,WHII93K X PYUQ,0.000042
209996,WHII93K X QBOL,0.000149
209997,WHII93K X RIBP,0.000013
209998,WHII93K X RVSZ,0.999677


In [31]:
df_answer.reset_index(drop=True, inplace=True)

In [32]:
df_answer.to_csv('finale_version_multiClass_7Models.csv', index=False)

In [33]:
from google.colab import files
files.download('finale_version_multiClass_7Models.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

OK